In [1]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings

In [2]:
from dotenv import load_dotenv, find_dotenv
import os
import openai
# Load environment variables from a .env file
load_dotenv(find_dotenv())

# Set the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n. Also if asked about a code that is not on the context
    provided just say, "answer is not available in the context".
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

model = OpenAI(openai_api_key=openai.api_key, model_name="gpt-3.5-turbo-instruct")

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

In [5]:
embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)

user_question = "cual es el codigo de la acondroplastia"

db = FAISS.load_local(folder_path="/teamspace/studios/this_studio/Curso_IA_Gen/Formularios",embeddings=embeddings,index_name="myFaissIndex",allow_dangerous_deserialization=True)
docs = db.similarity_search(user_question)

response = chain(
    {"input_documents": docs, "question": user_question}, return_only_outputs=True
)

print(response)

{'output_text': 'The code for acondroplasia (osteoesclerosis congénita) is Q77.4.'}


In [4]:
import gradio as gr

# Función para procesar la pregunta del usuario y generar la respuesta
def responder_cie(user_question):
    embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
    db = FAISS.load_local(folder_path="/teamspace/studios/this_studio/Curso_IA_Gen/Formularios",embeddings=embeddings,index_name="myFaissIndex",allow_dangerous_deserialization=True)
    docs = db.similarity_search(user_question)
    response = chain({"input_documents": docs, "question": user_question})
        
    return response['output_text']


# Crear la interfaz de Gradio
interface = gr.Interface(
   fn=responder_cie,
   inputs="text",
   outputs="text",
   title="Interfaz de Búsqueda",
   description="Introduce una pregunta para obtener la respuesta basada en los documentos proporcionados."
)
# Iniciar la interfaz
if __name__ == "__main__":
   interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7879
Running on public URL: https://6cb2438074852273e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
